# A script that for all images, crop boxes @ the bounding boxes coordinates given. BBx data is stored in .txt format

In [1]:
# Read images and their corresponding bounding boxes
# Crop images using their bounding boxes coordinates
# Using the label of each bounding box, save that croped box in a subfolder accordingly

In [1]:
import glob
import os 
import cv2 
import random
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
from PIL import Image
import pandas as pd

%matplotlib inline

In [2]:
train_box_dir = 'train_bbxs'
train_images_dir = 'train_images'

images_path = os.listdir(train_images_dir)
images_boundingboxs = os.listdir(train_box_dir)

print('Number of images:', len(images_path))
print('Number of images having bounding boxes:', len(images_boundingboxs))

Number of images: 5279
Number of images having bounding boxes: 4983


In [3]:
df = pd.read_csv('train_labels.csv')
df.head()

,img_name,label
0,00000.jpg,ا ظ ش 0 2 9
1,00001.jpg,ل ل 1
2,00002.jpg,ث ص ش 5 5
3,00003.jpg,ث غ د 2 5 2
4,00004.jpg,ش ف ب 3 3 4


In [4]:
df.iloc[1]['label']

'ل ل 1'

In [5]:
'''
Crop bounding boxes inside an image. bounding boxes data is stored in a txt file.

Params:
------
fname           string   the txt file that contains the coordinates of the bounding boxes
img_path        string   the path of the image to extract from the boxes
label           string   the label of the car plate characters for the given img
final_dir_path  string   the directory path to which save the charcaters subdirectories in
character_freq  dict     dictionary containing the frequency of cropped characters from the given img
verbose         bool     defualt false. If true, print some visualizations of the image and its cropped boxes

'''
def crop_img(fname, img_path, label, final_dir_path, character_freq={}, verbose=False):
    
    img = io.imread(img_path)
    if (verbose):
        plt.imshow(img)
        
    with open(fname, "r+") as file:
        bounding_boxs_data = np.array([[int(position) for position in line.split(', ')] for line in file])
        
        i = 1
        if (verbose):
            fig = plt.figure(figsize=(100, 100))
            img_boxes= img.copy()
        
        for xmin,ymin,xmax,ymax in bounding_boxs_data:
            # For boxes out of image dimensions
            xmin = 0 if (xmin<0) else xmin
            ymin = 0 if (ymin<0) else ymin
            xmax = img.shape[1] if (xmax>img.shape[1]) else xmax
            ymax = img.shape[0] if (ymax>img.shape[0]) else ymax

            # there are some boxes where xmin > xmax
            if (xmin >=xmax):
                continue
            if (ymin >=ymax):
                continue
                
            # Cropping the bounding box
            box = img[ymin:ymax,xmin:xmax, : ]
            box_img = Image.fromarray(box)
            # Getting the label corresponding to this box
            char_label = label.split()[i-1]
            subdir_path = os.path.join(final_dir_path,char_label)
            
            # If a directory for this label is not created before, create it
            if not os.path.exists(subdir_path):
                os.makedirs(subdir_path)
                character_freq[char_label] = 0
            
            box_img.save(os.path.join(subdir_path, str(character_freq[char_label])+'.jpg'))
            character_freq[char_label]+=1
            if (verbose):
                img_with_bbxs = cv2.rectangle(img_boxes,(xmin, ymin), (xmax, ymax), (0,255,0))
                fig.add_subplot(50,50, i)
                plt.imshow(img[ymin:ymax,xmin:xmax, : ])
            i += 1
    if (verbose):
        plt.show()
        plt.imshow(img_with_bbxs)
        plt.show()

In [6]:
character_freq={}

for indx in range(0, len(images_boundingboxs)):

    img_num = images_boundingboxs[indx][:-4]
    fname = os.path.join(train_box_dir, images_boundingboxs[indx])
    img_path = os.path.join(train_images_dir, img_num +'.jpg')

    crop_img(fname=fname, img_path=img_path, label=df.iloc[int(img_num)]['label'], verbose=False, final_dir_path='./cropped_characters', character_freq=character_freq)

In [9]:
character_freq

{'ا': 418,
 'ظ': 493,
 'ش': 450,
 '0': 1250,
 '2': 1273,
 '9': 1316,
 'ل': 463,
 '1': 1312,
 'ث': 443,
 'ص': 418,
 '5': 1218,
 'غ': 445,
 'د': 446,
 'ف': 443,
 'ب': 442,
 '3': 1250,
 '4': 1210,
 'ت': 446,
 '6': 1215,
 'ع': 452,
 'م': 469,
 'س': 440,
 'ذ': 428,
 '7': 1215,
 'خ': 488,
 'ك': 443,
 'ه': 416,
 'ق': 472,
 'ج': 435,
 'ض': 422,
 '8': 1234,
 'ز': 430,
 'و': 452,
 'ح': 440,
 'ر': 478,
 'ى': 418,
 'ن': 478,
 'ط': 441}